In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

- I want the ML to take as input 17 parameters corresponding to $[ \{ \alpha_i \}, \Omega_m, \Omega_m h^2 ]$ and to output the luminosity distance of the SN as a function of z, i.e. $d_L(z)$.
- In practice what I want is the ML to output an array of $d_L$, one of every bin in z. Therefore: $input = 1 \times 17$ and $output=1 \times N_{zbins}$.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
## Here I define an affine layer which will take care of the data normalization. 

class Affine(nn.Module):
    def __init__(self):
        super(Affine,self).__init__
        self.gain = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return self.gain * x + self.bias
        

In [9]:
## Residual block - following the diagram in original ref: https://arxiv.org/pdf/1512.03385
## Each block will have two linear layers.
## The second activation is applied after I sum with the skip connection: ACT( F(x) + x )
class ResBlock(nn.Module):
    def __init__(self, in_size, out_size):
        if in_size != out_size:
            self.skip = nn.Linear(in_size, out_size, bias=False)
        else:
            self.skip = nn.Identity()
            
        self.linear1 = nn.Linear(in_size, out_size)
        self.linear2 = nn.Linear(out_size, out_size)

        self.norm1 = Affine()
        self.norm2 = Affine()

        self.act1 = nn.Relu()
        self.act2 = nn.Relu()
        
    def forward(self,x):
        xskip = self.skip(x)
        x = self.act1(self.linear1(self.norm1(x)))
        x = self.linear2(self.norm2(x))
        out = self.act2(x + xskip)
        return out
        
        

In [ ]:
class ResMLP(nn.Module):
    def __init__(self, input_dim, output_dim, block_nums):
        super(ResMLP,self).__init__()
        
        self.block = ResBlock()
        # Pytorch list that saves the different layers. These layers are not connected in a NN yet.
        self.modules = nn.ModuleList()
        # Activation function to use
        self.act = nn.Relu()
        
        # Write a for loop that controls how many ResBlocks I include in my full network
        for i in range(block_nums):
            self.modules.append(self.block(input_dim,input_dim))
            
        # The last layer I append in the nn.ModuleList is the fully connected linear layer (output layer of my NN)
        self.modules.append(nn.Linear(input_dim, output_dim))
        
        def forward(x,output_dim):
            ## I need to add one layer here to embed my input vector to the bigger internal space 
            # Connect the different blocks in the NN
            for block in self.modules[:-1]:
                x = self.act(block(x))
            # Pass the output through the final fully connected linear layer
            out = self.modules[-1](x)
            
            return out               
        
        
        
    
    